In [47]:
# Jupyter Notebook Chapter 11 Exercises Program
# DSC 530
# Week 9
# EDA Assignment Week 9
# David Berberena
# 2/11/2024

# Program Start

# Importing of needed libraries and data to establish the correct coding environment

from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")
import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

import statsmodels.formula.api as smf

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz")

import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [48]:
# Exercise 11-1

# Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of 
# birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best 
# prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the 
# people in the pool.

# In order to figure this out, we would need to see the effect of all variables on pregnancy length (prglngth). 
# The provided Exercise functions allows us to loop through all variables, compute the statistical significance value 
# (R-Squared) against prglngth, and list the top variables with the highest significance value.

# Function code is taken from the Exercise file to fulfill assignment requirements (function has been changed to account
# for pregnancy length variable)

import patsy

def GoMining(df):

    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ agepreg + ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables

            

In [49]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

# GoMining() is used to return all variable names within the live dataset with their significance values
variables = GoMining(live)
variables

[(5.461177492105396e-05, 'caseid'),
 (0.0006488168245175618, 'pregordr'),
 (0.002249407606621512, 'pregend1'),
 (0.0045801411639412315, 'nbrnaliv'),
 (0.0004105037064817685, 'cmprgend'),
 (2.4817063965953956e-05, 'cmprgbeg'),
 (0.0016590163592545837, 'gestasun_m'),
 (0.0010938126121891045, 'gestasun_w'),
 (0.8065115675039273, 'wksgest'),
 (0.09562434659774721, 'mosgest'),
 (1.8878682399026125e-05, 'bpa_bdscheck1'),
 (0.00021153249870431434, 'babysex'),
 (0.12067945355791998, 'birthwgt_lb'),
 (0.0002500814672437013, 'birthwgt_oz'),
 (0.0004105037064817685, 'cmbabdob'),
 (2.0901979782106395e-05, 'kidage'),
 (3.0251830422600712e-05, 'hpagelb'),
 (0.0014368115355252176, 'matchfound'),
 (0.002918483981364295, 'anynurse'),
 (0.0005402902882050142, 'frsteatd_n'),
 (0.0013304832813814116, 'frsteatd_p'),
 (0.0002130347575965974, 'frsteatd'),
 (0.002047727311564329, 'cmlastlb'),
 (0.00017932809415766027, 'cmfstprg'),
 (0.0012940540023294034, 'cmlstprg'),
 (0.0008906016746064171, 'cmintstr'),
 (0

In [50]:
# Function code is taken from the Exercise file to fulfill assignment requirements

import re

def ReadVariables():

    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = pd.concat([vars1, vars2])
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=77):

    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)
            
# MiningReport() is called to display the 77 variables with positive R-Squared values and a description of what the 
# variables represent to glean whether the mother's coworkers would be able to know this information prior to the betting

MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8065115675039273 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.7772278070905592 AGE AT TIME OF CONCEPTION
totalwgt_lb 0.12550330803309884
birthwgt_lb 0.12067945355791998 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10383887474318443 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562434659774721 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.02241842075365197 PRGLNGTH IMPUTATION FLAG
nbrnaliv 0.0045801411639412315 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
anynurse 0.002918483981364295 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.00256701215878008 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249407606621512 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
cmlastlb 0.002047727311564329 CM FOR R'S MOST RECENT LIVE BIRTH
fmarcon5_i 0.00197180591645274 FMARCON5 IMPUTATION FLAG
evuseint 0.0019247289345255547 EG-1 USE ANY METHOD IN PREGNANCY INTERVAL?

In [52]:
# Looking at the variables and their corresponding meanings, it seems that the only variables that the mother's coworkers 
# could possibly know at the time the predictions and betting take place are nbrnaliv, birthord, and race. By 30 weeks into
# the pregnancy, the coworkers can assume the baby born will be alive. The order of birth is automatically assumed to be one
# due to the non-existence of a twin pregnancy or higher. Race is able to be considered as a variable as anyone working in 
# the company can look into the mother's work profile and discover her race (if she has not already identified herself to 
# her coworkers already). All of the other variables within the live dataset are questionable as to whether the others 
# betting on the child's date of birth would have access to such information at the 30 week pregnancy mark.

In [53]:
# Exercise 11-3

# If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels 
# with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how many 
# children a woman has born; in the NSFG dataset, this variable is called numbabes.

# Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000.
# How many children would you predict she has born?

# This question is straightforward in the sense that it tells us what variables to put into the formula to conduct a Poisson
# regression to realize the outcome of the number of children predicted by the model. Age is represented by the variable 
# "age_r" in the join dataset, race is represented by the "race" variable, being a college graduate is denoted with the 
# "educat" variable, and an annual household income above $75,000 is shown by the "totincr" variable. The "numbabes" 
# variable is the outcome variable, the model needed is a Poisson regression model, and the dataset is the join dataset
# established earlier. The summary of the results is displayed to indicate that there is no highly correlated variables.

formula = 'numbabes ~ age_r + race + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.689226
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8879
Method:                           MLE   Df Model:                            4
Date:                Tue, 06 Feb 2024   Pseudo R-squ.:                 0.02984
Time:                        09:38:39   Log-Likelihood:                -15007.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.519e-198
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1545      0.049     23.661      0.000       1.059       1.250
age_r          0.0206      0.001     20.316      0.000       0.019       0.023
race          -0.0802      0.011     -7.043      0.000      -0.102      -0.058
totincr       -0.0198      0.002    -10.547      0.000      -0.023      -0.016
educat        -0.0444      0.003    -15.162      0.000      -0.050      -0.039
==============================================================================
"""

In [54]:
# This prediction results code is taken from the Exercise file and modified to fit the parameters of the current exercise. 
# Each column is denoted as a predictor variable, and the Pandas dataframe is used to construct the specific observations 
# (35 is the age needed, 1 is the desgination for Black/African American in the race variable, 14 is the designation for the
# annual household income threshold of higher than $75,000 in the totincr variable, and 16 is the observation which denotes
# the number of years completed in school (16 equals an undergrad college graduate) in the educat variable) needed as called
# for by the exercise guidelines. These observation scales were found by using a combination of the previous mining 
# functions and summary statistics.

columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results.predict(new)

# The outcome is 2.24 children, which rounds down to 2 children.

0    2.244372
dtype: float64

In [55]:
# Exercise 11-4

# If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in 
# StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, 
# cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable
# called rmarital.

# Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about 
# $45,000. What is the probability that she is married, cohabitating, etc?

# We will provide multinomial logistic regression summary results using the parameters set forth by the exercise. The 
# outcome variable is rmarital, the predictors are age, race, annual household income, and education. The type of model is 
# a mnlogit model, and the dataset is still the join dataset.

formula='rmarital ~ age_r + race + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.100457
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8859
Method:                           MLE   Df Model:                           20
Date:                Tue, 06 Feb 2024   Pseudo R-squ.:                  0.1557
Time:                        09:38:47   Log-Likelihood:                -9776.5
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.7129      0.305     15.465      0.000       4.116       5.310
age_r         -0.0563      0.006     -9.685      0.000      -0.068      -0.045
race          -0.4868      0.067     -7.270      0.000      -0.618      -0.356
totincr       -0.1384      0.011    -12.098      0.000      -0.161      -0.116
educat        -0.2035      0.019    -10.965      0.000      -0.240      -0.167
------------------------------------------------------------------------------
rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.6351      0.971     -4.774      0.000      -6.538      -2.732
age_r          0.1305      0.019      6.863      0.000       0.093       0.168
race          -0.0856      0.169     -0.505      0.614      -0.418       0.246
totincr       -0.3316      0.032    -10.439      0.000      -0.394      -0.269
educat        -0.1015      0.048     -2.114      0.035      -0.196      -0.007
------------------------------------------------------------------------------
rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.3550      0.324     -7.267      0.000      -2.990      -1.720
age_r          0.0640      0.007      9.785      0.000       0.051       0.077
race          -0.3083      0.069     -4.500      0.000      -0.443      -0.174
totincr       -0.2273      0.012    -19.696      0.000      -0.250      -0.205
educat         0.0676      0.017      4.055      0.000       0.035       0.100
------------------------------------------------------------------------------
rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0714      0.372      2.882      0.004       0.343       1.800
age_r          0.0072      0.007      1.003      0.316      -0.007       0.021
race          -0.5091      0.079     -6.421      0.000      -0.664      -0.354
totincr       -0.3022      0.014    -20.980      0.000      -0.330      -0.274
educat        -0.0127      0.021     -0.599      0.549      -0.054       0.029
------------------------------------------------------------------------------
rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.4843      0.296     25.261      0.000       6.904       8.065
age_r         -0.0899      0.006    -15.973      0.000      -0.101      -0.079
race          -1.5675      0.065    -24.037      0.000      -1.695      -1.440
totincr       -0.3014      0.011    -26.346      0.000      -0.324      -0.279
educat        -0.0747      0.017     -4.279      0.000      -0.109      -0.040
==============================================================================
""

In [56]:
# Just like the previous exercise, the prediction code is taken and adjusted to output the probability value predicted by 
# the independent variables in question. 25 is the age needed, 2 is the desgination for white in the race variable, 11 is 
# the designation for the annual household income threshold of $45,000 in the totincr variable, and 12 is the observation 
# which denotes the number of years completed in school (12 equals a higg school graduate). 

columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.712204,0.139107,0.001173,0.033348,0.02781,0.086358


In [57]:
# As seen above, the probability that the woman is married is 71 percent. The other probability value that is significant is
# that there is almost a 14 percent chance that she is cohabitating. 